# Plots of SFH for Outshined/not Outshined Galaxies

In [ ]:
import astropy.units as u
import numpy as np
import os
from astropy.table import Table
import copy
from tqdm import tqdm
os.environ["PIXEDFIT_HOME"] = "/nvme/scratch/work/tharvey/piXedfit/"
from EXPANSE import ResolvedGalaxy, ResolvedGalaxies
from matplotlib import pyplot as plt
import matplotlib.patheffects as pe
import glob
from scipy import signal
from scipy.interpolate import interp1d
from scipy.stats import binned_statistic
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.cm as cm
from cmasher import take_cmap_colors
import cmasher as cmr

# Change dpi to make plots larger

plt.rcParams["figure.dpi"] = 100

# Disable tex in matplotlib

plt.rcParams["text.usetex"] = False
%matplotlib inline

### Initial Loading of the Galaxy Objects

In [ ]:
galaxies = ResolvedGalaxies(
    ResolvedGalaxy.init_all_field_from_h5("JOF_psfmatched", n_jobs = 6)
)

Choosing a sample to plot. 

In [ ]:
ids = [10536, 9634, 10045, 15021, 5371]
ids = [str(i) for i in ids]


test_galaxies = galaxies.filter_IDs(ids)

ids = [10130, 12816, 2994, 845, 615, 7687]
ids = [str(i) for i in ids]

control_galaxies = galaxies.filter_IDs(ids)

Plotting the SFH plot for the different models.

In [ ]:

fig = plt.figure(figsize=(4, 10), dpi = 200, tight_layout = True, facecolor = 'white')

axes = fig.subplots(len(control_galaxies), 1, sharex=True)

resolved_runs = ['CNST_SFH_RESOLVED', 'CNST_SFH_RESOLVED_NOMIN', 'CNST_SFH_RESOLVED_VORONOI']
resolved_run_names = ['Resolved (piXedfit)', 'Resolved (piXedfit, no min)', 'Resolved (voronoi)']
# nice color palette with distinct colors

resolved_cmap = 'cmr.apple'
resolved_colors = take_cmap_colors(resolved_cmap, len(resolved_runs), cmap_range=(0, 0.8))
resolved_colors = ['mediumseagreen', 'gold', 'orangered']
resolved_color_dict = dict(zip(resolved_runs, resolved_colors))

integrated_runs = ['photoz_lognorm', 'photoz_delayed', 'photoz_continuity', 'photoz_cnst', 'photoz_continuity_bursty']
integrated_run_names = ['Lognorm SFH', r'Delayed-$\tau$ SFH', 'Continuity SFH', 'Constant SFH', 'Bursty SFH']
# uniuqe cmap - not viridis
integrated_cmap = 'cmr.guppy'
# get colors from cmap
cmap = plt.get_cmap(integrated_cmap)
integrated_colors = [cmap(i) for i in np.linspace(0, 1, len(integrated_runs))]
integrated_color_dict = dict(zip(integrated_runs, integrated_colors))
resolved_run = "CNST_SFH_RESOLVED_VORONOI"
print(len(test_galaxies), len(axes))
for i, galaxy in enumerate(control_galaxies):
    print(i)
    ax = axes[i]

    for integrated_run, integrated_color in zip(integrated_runs, integrated_colors):
        _, _ = galaxy.plot_bagpipes_sfh(run_name = integrated_run, axes = ax, bins_to_show=['TOTAL_BIN'], fig = fig, marker_colors = [integrated_color], time_unit='yr', linestyle = 'dashdot')
    
    
    for resolved_runa, resolved_color in zip(resolved_runs, resolved_colors):
        try:
            _, _ = galaxy.plot_bagpipes_sfh(run_name = resolved_runa, axes = ax, bins_to_show=['RESOLVED'], fig = fig, resolved_color=resolved_color, time_unit='yr', zorder=10)
        except:
            pass

    # Delete legend
    ax.get_legend().remove()

    ax.set_xscale('log')
    ax.set_xlabel('')
    # Label with galaxy ID and redshift

    ax.text(0.05, 0.95, f'{galaxy.galaxy_id}\nz={galaxy.redshift:.2f}\n$N_{{bins}}={galaxy.get_number_of_bins(binmap_type="voronoi")}$', transform=ax.transAxes, ha='left', va='top', zorder=30, fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')])
    
    # Write resolved stellar mass 

    # Get min, max mass from table
    min_mass = (1e10, 0, 0)
    max_mass = (0, 0, 0)
    for run in galaxy.sed_fitting_table['bagpipes'].keys():
        if run == 'photoz_dpl':
            continue
        
        if 'TOTAL_BIN' in galaxy.sed_fitting_table['bagpipes'][run]['#ID']:
            mass = galaxy.sed_fitting_table['bagpipes'][run]['stellar_mass_50'][0]
            upper = galaxy.sed_fitting_table['bagpipes'][run]['stellar_mass_84'][0] - mass
            lower = mass - galaxy.sed_fitting_table['bagpipes'][run]['stellar_mass_16'][0]

            if mass > max_mass[0]:
                max_mass = (mass, upper, lower, run)
            elif mass < min_mass[0]:
                min_mass = (mass, upper, lower, run)

    ax.text(0.97, 0.75, f'$\log_{{10}}(M_{{\star, \mathrm{{integrated, max}}}}) = {max_mass[0]:.2f}^{{+{max_mass[1]:.2f}}}_{{-{max_mass[2]:.2f}}}$', transform=ax.transAxes, ha='right', va='top', fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')], color=integrated_color_dict[max_mass[3]],  zorder=30)
    ax.text(0.97, 0.85, f'$\log_{{10}}(M_{{\star, \mathrm{{integrated, min}}}}) = {min_mass[0]:.2f}^{{+{min_mass[1]:.2f}}}_{{-{min_mass[2]:.2f}}}$', transform=ax.transAxes, ha='right', va='top', fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')], color=integrated_color_dict[min_mass[3]],  zorder=30)


    val = galaxy.resolved_mass[resolved_run][1]
    upper = galaxy.resolved_mass[resolved_run][2] - val
    lower = val - galaxy.resolved_mass[resolved_run][0]

    ax.text(0.97, 0.95, f'$\log_{{10}}(M_{{\star, \mathrm{{resolved}}}}) = {val:.2f}^{{+{upper:.2f}}}_{{-{lower:.2f}}}$', transform=ax.transAxes, ha='right', va='top', fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')], color=resolved_color_dict[resolved_run],  zorder=30)

    ax.set_xlim(1e6, 5e8)

# Add a dummy legend with SFHs

axes[-1].set_xlabel('Lookback Time [yr]')

points = []
for pos, (resolved_run, resolved_color) in enumerate(zip(resolved_run_names, resolved_colors)):
    point = ax.plot([], [], color = resolved_color, label = resolved_run)
    points.append(point[0])

for pos, (integrated_run, integrated_color) in enumerate(zip(integrated_run_names, integrated_colors)):
    point = ax.plot([], [], color = integrated_color, label = integrated_run, linestyle = 'dashdot')
    points.append(point[0])

fig.legend(handles = points, loc = 'lower center', fontsize = 8, title = 'SFH Type', title_fontsize = 8, markerscale = 0.7, labelspacing=0.1, ncol=2, bbox_to_anchor=(0.5, -0.07))

fig.savefig('/nvme/scratch/work/tharvey/EXPANSE/plots/sfh_comparison_no_outshined.pdf', dpi = 200, facecolor = 'white', bbox_inches = 'tight')


In [ ]:

resolved_runs = ['CNST_SFH_RESOLVED', 'CNST_SFH_RESOLVED_NOMIN', 'CNST_SFH_RESOLVED_VORONOI']
resolved_run_names = ['Resolved (piXedfit)', 'Resolved (piXedfit, no min)', 'Resolved (voronoi)']
# nice color palette with distinct colors

resolved_cmap = 'cmr.apple'
resolved_colors = take_cmap_colors(resolved_cmap, len(resolved_runs), cmap_range=(0, 0.8))
resolved_colors = ['mediumseagreen', 'gold', 'orangered']
resolved_color_dict = dict(zip(resolved_runs, resolved_colors))

integrated_runs = ['photoz_lognorm', 'photoz_delayed', 'photoz_continuity', 'photoz_cnst', 'photoz_continuity_bursty']
integrated_run_names = ['Lognorm SFH', r'Delayed-$\tau$ SFH', 'Continuity SFH', 'Constant SFH', 'Bursty SFH']
# uniuqe cmap - not viridis

resolved_masses = {}
for galaxy in galaxies:
    vals = []
    for resolved_run in resolved_runs:
        if galaxy.resolved_mass is not None and resolved_run in galaxy.resolved_mass.keys():
            val = galaxy.resolved_mass[resolved_run][1]
            vals.append(val)
    if len(vals) > 0:
        resolved_masses[galaxy.galaxy_id] = np.nanmax(vals)

integrated_masses = {}
for galaxy in galaxies:
    vals = []
    for integrated_run in integrated_runs:
        if integrated_run in galaxy.sed_fitting_table['bagpipes'].keys():
            if 'TOTAL_BIN' in galaxy.sed_fitting_table['bagpipes'][integrated_run]['#ID']:
                val = galaxy.sed_fitting_table['bagpipes'][integrated_run]['stellar_mass_50'][0]
                vals.append(val)
    if len(vals) > 0:
        integrated_masses[galaxy.galaxy_id] = np.nanmin(vals)

counter = 0

ids = []
# Look for IDs where max resolved mass is close to (within 0.1 dex) of min integrated mass
for galaxy_id, resolved_mass in resolved_masses.items():
    if (integrated_masses[galaxy_id] - resolved_mass) > -0.1 and resolved_mass < 9:
        print(galaxy_id, resolved_mass, integrated_masses[galaxy_id])
        counter += 1
        ids.append(galaxy_id)

print(counter)




In [ ]:
control_galaxies_all = galaxies.filter_IDs(ids)

In [ ]:
for galaxy in control_galaxies_all:
    galaxy.pixel_by_pixel_galaxy_region(snr_req=2, band_req='all_wide_nobreak', mask='detection')
    galaxy.pixel_by_pixel_binmap(galaxy.gal_region['SNR_2_all_wide_nobreak'])

In [ ]:
nbins = 0
for galaxy in control_galaxies_all:
    nbins += galaxy.get_number_of_bins(binmap_type="pixel_by_pixel")

print(nbins)

## Dense Basis Testing


In [ ]:
db_atlas_path = f'/nvme/scratch/work/tharvey/EXPANSE/scripts/pregrids/db_atlas_JOF_10000_Nparam_3.dbatlas'
from EXPANSE.dense_basis import get_priors
priors = get_priors(db_atlas_path)
#galaxy = ResolvedGalaxy.init_from_h5('/nvme/scratch/work/tharvey/EXPANSE/galaxies/JOF_psfmatched_3217.h5')


In [ ]:
resolved_mass = galaxy.get_total_resolved_property('db_atlas_JOF_star_stack_pixedfit_zphotoz_delayed', 
                                    sed_fitting_tool='dense_basis',
                                    property='mstar')

table = galaxy.sed_fitting_table['dense_basis']['db_atlas_JOF_star_stack_pixedfit_zphotoz_delayed']
mask = table['#ID'] == 'TOTAL_BIN'
integrated_mass = table['mstar_50.0'][mask][0]

print(resolved_mass, integrated_mass)


In [ ]:
binmap_type = 'voronoi'
db_atlas_path = f'/nvme/scratch/work/tharvey/EXPANSE/scripts/pregrids/db_atlas_JOF_10000_Nparam_3.dbatlas'

ok_galaxies = galaxies.filter_single_bins(binmap=binmap_type)

for galaxy in tqdm(ok_galaxies):
    fit_results = galaxy.run_dense_basis(db_atlas_path, plot=True, fit_photometry='TOTAL_BIN+bin',
                                        fix_redshift='photoz_delayed', binmap_type=binmap_type, use_emcee=False,
                                        priors = priors, n_jobs=6, save_outputs=True, overwrite=True,
                                        save_full_posteriors=False)

In [ ]:
print(galaxy.sed_fitting_table['dense_basis']['db_atlas_JOF_star_stack_pixedfit_zphotoz_delayed'])

In [ ]:
np.unique(galaxies[1].pixel_by_pixel_map)
galaxies[1].get_number_of_bins(binmap_type='pixel_by_pixel')

In [ ]:
remove = []
total_bins = 0
for galaxy in galaxies:
    galaxy.pixel_by_pixel_galaxy_region(snr_req=2, band_req='all_wide_nobreak', mask='detection', overwrite=True)
    galaxy.pixel_by_pixel_binmap(galaxy.gal_region['SNR_2_all_wide_nobreak'], overwrite=True)
    if galaxy.get_number_of_bins(binmap_type = 'pixel_by_pixel') == 0:
        #print(f"Galaxy {galaxy.galaxy_id} has no bins")
        remove.append(galaxy.galaxy_id)
    
    else:
        galaxy.measure_flux_in_bins(binmap_type='pixel_by_pixel', overwrite=True)
        nbins = galaxy.get_number_of_bins(binmap_type='pixel_by_pixel')
        total_bins += nbins

print(len(remove), len(galaxies))
print(total_bins)

In [ ]:
nbins = []
skipped = []
zmax = 0
for galaxy in galaxies:
    n = galaxy.get_number_of_bins(binmap_type='voronoi')
    if n <= 1:
        skipped.append(galaxy.galaxy_id)
    else:
        if galaxy.redshift > zmax:
            zmax = galaxy.sed_fitting_table['bagpipes']['photoz_delayed']['redshift_50'][0]
            zmax_id = galaxy.galaxy_id
        nbins.append(n)

print(np.median(nbins), np.mean(nbins), np.std(nbins), np.max(nbins), np.min(nbins), np.sum(nbins))
print(len(skipped))
print(zmax, zmax_id)
    

In [ ]:
galaxy_high = galaxies.filter_IDs([zmax_id])[0]

galaxy_high.plot_bagpipes_overview(bagpipes_runs={'CNST_SFH_RESOLVED_VORONOI':['RESOLVED']}, binmap_type='voronoi')

In [ ]:
binmap = 'voronoi'
db_atlas_path = f'/nvme/scratch/work/tharvey/EXPANSE/scripts/pregrids/db_atlas_JOF_10000_Nparam_3.dbatlas'
db_atlas_path = f'/nvme/scratch/work/tharvey/EXPANSE/scripts/pregrids/db_atlas_JOF_10000_Nparam_3.dbatlas'
from EXPANSE.dense_basis import get_priors
priors = get_priors(db_atlas_path)
#galaxy = ResolvedGalaxy.init_from_h5('/nvme/scratch/work/tharvey/EXPANSE/galaxies/JOF_psfmatched_3217.h5')


plot_galaxies = galaxies.filter_single_bins(binmap)

fig = plt.figure(figsize=(8, 5), dpi=200, tight_layout=True, facecolor='white')

# Make a gridspec with one large square plot on the left and three smaller rectangular plots on the right

from matplotlib.gridspec import GridSpec

gs = GridSpec(3, 2, figure=fig)

ax = fig.add_subplot(gs[:, 0])

sfh_axs = [fig.add_subplot(gs[i, 1]) for i in range(3)]


# Add ticks on right and top
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

ax.minorticks_on()
ax.tick_params(axis='both', direction='in', which='both')

cax = make_axes_locatable(ax).append_axes('top', size='5%', pad=0.05)

cmap = cmr.guppy
norm = plt.Normalize(4.5, 12)

data = {}
for galaxy in plot_galaxies:
    try:
        resolved_mass = galaxy.get_total_resolved_property(f'db_atlas_JOF_star_stack_{binmap}_zphotoz_delayed', 
                                            sed_fitting_tool='dense_basis',
                                            property='mstar')
        table = galaxy.sed_fitting_table['dense_basis'][f'db_atlas_JOF_star_stack_{binmap}_zphotoz_delayed']
        mask = table['#ID'] == 'TOTAL_BIN'
        integrated_mass = table['mstar_50'][mask][0]
        lower_error = integrated_mass - table['mstar_16'][mask][0]
        upper_error = table['mstar_84'][mask][0] - integrated_mass
        redshift = table['z_50'][mask][0]
        data[galaxy.galaxy_id] = [integrated_mass, resolved_mass]

        #ax.text(integrated_mass, resolved_mass-0.2, galaxy.galaxy_id, fontsize=6, ha='center', va='center', color='black', zorder=5, path_effects=[pe.withStroke(linewidth=1, foreground='white')])
        ax.scatter(integrated_mass, resolved_mass, color=cmap(norm(redshift)), label=galaxy.galaxy_id, zorder=4, s=10, edgecolors='black', linewidths=1)
        ax.errorbar(integrated_mass, resolved_mass, xerr=[[lower_error], [upper_error]], marker='none', alpha=0.5, color=cmap(norm(redshift)), zorder=2, linewidth=1)
    except Exception as e:
        print(e)
        print(galaxy.galaxy_id)
        print('crash!')
        pass


ids_to_plot_sfh = [12443, 2808, 9350] #10130, 7392
colors = ['blue', 'green', 'red']
filter_galaxies = galaxies.filter_IDs([str(i) for i in ids_to_plot_sfh])
for i, galaxy in enumerate(filter_galaxies):
    # pair of nice contrasting colors for SFH
    nice_colors = ['dodgerblue', 'orangered']
    if galaxy.sed_fitting_sfhs is None: 
            fit_results = galaxy.run_dense_basis(db_atlas_path, plot=True, fit_photometry='TOTAL_BIN+bin',
                                        fix_redshift='photoz_delayed', binmap_type=binmap, use_emcee=False,
                                        priors = priors, n_jobs=6, save_outputs=True, save_sfh=True, overwrite=True,
                                        save_full_posteriors=False)
    galaxy.plot_internal_sfh(ax=sfh_axs[i], sed_fitter='dense_basis',
                            run_name = 'db_atlas_JOF_star_stack_voronoi_zphotoz_delayed',
                            bins_to_show=['RESOLVED', 'TOTAL_BIN'], colors=nice_colors)
    if i == 0:
        handles, labels = sfh_axs[i].get_legend_handles_labels()
        # replace 'TOTAL_BIN' with 'Integrated'
        labels = [label.replace('TOTAL_BIN', 'Integrated') for label in labels]
        labels = [label.replace('RESOLVED', 'Resolved') for label in labels]

        leg = sfh_axs[i].legend(loc='center', fontsize=7, title='SFH Type', title_fontsize=9, markerscale=0.7, labelspacing=0.1, handles = handles, labels = labels, bbox_to_anchor=(0.5, 1.25))
    if i != len(filter_galaxies) - 1:
        sfh_axs[i].set_xlabel('')
    # Label with redshift and galaxy ID
    sfh_axs[i].text(0.05, 0.95, f'{galaxy.galaxy_id}\nz={galaxy.redshift:.2f}\n$N_{{bins}}={galaxy.get_number_of_bins(binmap_type="voronoi")}$', transform=sfh_axs[i].transAxes, ha='left', va='top', zorder=30, fontsize=8, path_effects=[pe.withStroke(linewidth=1, foreground='white')], color=colors[i])
    # Connect point on main plot with SFH plot - put a box around the point
    ax.scatter(data[galaxy.galaxy_id][0], data[galaxy.galaxy_id][1], zorder=5, s=40, edgecolors=colors[i], linewidths=1.5, facecolors='none', marker = 's')

# add a 1:1 line

fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax, orientation='horizontal', label='Redshift')
cax.xaxis.set_ticks_position('top')
cax.xaxis.set_label_position('top')

ax.plot([6, 11], [6, 11], color='black', linestyle='--', linewidth=1)
ax.set_xlabel(r'Integrated Mass [$\log_{{10}}(M_\star/ M_\odot)$]')
ax.set_ylabel(r'Resolved Mass [$\log_{{10}}(M_\star/ M_\odot)$]')

ax.text(0.02, 0.97, 'Dense Basis SED Fitting', va='top', ha='left', fontsize=10, color='black', transform=ax.transAxes)
ax.text(0.02, 0.93, f'{binmap} binning', va='top', ha='left', fontsize=10, color='black', transform=ax.transAxes)
plt.show()

fig.savefig(f'../plots/mass_comparison_db_{binmap}.pdf', dpi=300)



### Reload PXP properties


In [ ]:
binmap_type = 'pixel_by_pixel'
ids = [
    "10092",
    #"10130",
    #"10161",
    "10376",
    "10816",
    "10896",
    "11482",
    "11600",
    "12050",
    "12488",
    "12779",
    "12816",
    "13892",
    "1623",
    "1951",
    "2015",
    "2439",
    "2643",
    "2994",
    "3398",
    "3434",
    "3845",
    "3908",
    "4234",
    "4266",
    "531",
    "6074",
    "615",
    "6744",
    "7155",
    "7526",
    "7687",
    "845",
    "8854",
    "906",
    "9078",
]

pxp_galaxies = galaxies.filter_IDs(ids)

remove = []
for galaxy in pxp_galaxies:
    galaxy.pixel_by_pixel_galaxy_region(snr_req=2, band_req='all_wide_nobreak', mask='detection')
    galaxy.pixel_by_pixel_binmap(galaxy.gal_region['SNR_2_all_wide_nobreak'])
    if galaxy.get_number_of_bins(binmap_type) == 0:
        print(f"Galaxy {galaxy.galaxy_id} has no bins")
        remove.append(galaxy.galaxy_id)
    else:
        galaxy.measure_flux_in_bins(binmap_type=binmap_type, overwrite=False)


galaxies_lowmass = pxp_galaxies.filter_IDs(remove, invert=True)

In [ ]:
run_name = 'CNST_SFH_RESOLVED_PBP'
properties_to_load=["stellar_mass", "sfr", "sfr_10myr"]
run_dir = '/nvme/scratch/work/tharvey/EXPANSE/pipes'

for galaxy in galaxies_lowmass:
    galaxy.load_bagpipes_results(run_name, meta={"binmap_type": "pixel_by_pixel"}, run_dir = run_dir)
    galaxy.get_resolved_bagpipes_sed(run_name, force=True, run_dir = run_dir, overwrite=True)
    # Save the resolved SFH
    galaxy.plot_bagpipes_sfh(
        run_name,
        bins_to_show=["RESOLVED"],
        plot=False,
        force=True,
        run_dir=run_dir,
    )

    # Save the resolved properties
    for prop in properties_to_load:
        galaxy.get_total_resolved_property(
            run_name,
            property=prop,
            log=prop == "stellar_mass",
            force=True,
            correct_map_band='F444W',
            correct_map=['detection', 'SNR_2_all_wide_nobreak'],
            run_dir=run_dir,
            overwrite=True,
        )
    



In [ ]:
galaxies_lowmass[0].psf_matched_data["star_stack"]['F444W'][galaxy.gal_region['detection'].astype(bool)]

In [ ]:
pxp_name = 'CNST_SFH_RESOLVED_PBP'
resolved_name = 'CNST_SFH_RESOLVED_VORONOI'

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)


for galaxy in galaxies_lowmass:

    pxp_mass = galaxy.get_total_resolved_property(pxp_name, property='stellar_mass')
    vor_mass = galaxy.get_total_resolved_property(resolved_name, property='stellar_mass')
    #vor_mass = galaxy.sed_fitting_table['bagpipes'][resolved_name]['stellar_mass_16'][0], galaxy.sed_fitting_table['bagpipes'][resolved_name]['stellar_mass_50'][0], galaxy.sed_fitting_table['bagpipes'][resolved_name]['stellar_mass_84'][0]
    p16, p50, p84 = pxp_mass
    v16, v50, v84 = vor_mass
    ax.scatter(p50, v50, color='black', label=galaxy.galaxy_id, zorder=4, s=10, edgecolors='black', linewidths=1)
    ax.errorbar(p50, v50, xerr=[[p50-p16], [p84-p50]], yerr=[[v50-v16], [v84-v50]], marker='none', alpha=0.5, color='black', zorder=2, linewidth=1)
    ax.text(p50, v50, galaxy.galaxy_id, fontsize=8, ha='right', va='bottom')
# Add 1:1

ax.set_xlabel(r'Pixel-by-Pixel Mass [$\log_{{10}}(M_\star/ M_\odot)$]')
ax.set_ylabel(r'Resolved Mass [$\log_{{10}}(M_\star/ M_\odot)$]')

ax.plot([6, 9.5], [6, 9.5], color='black', linestyle='--', linewidth=1)

    

In [ ]:
galaxy_id = '8854'

# Plot SFH
galaxy = ResolvedGalaxy.init_from_h5(f'/nvme/scratch/work/tharvey/EXPANSE/galaxies/JOF_psfmatched_{galaxy_id}.h5')
fig, ax = plt.subplots(1, 1, figsize=(7, 4), dpi=200)

#galaxy = galaxies.filter_IDs([galaxy_id])[0]

galaxy.plot_bagpipes_sfh(run_name='CNST_SFH_RESOLVED_VORONOI', bins_to_show=['RESOLVED'], fig=fig, axes=ax, resolved_color='red', time_unit='yr', zorder=10)
galaxy.plot_bagpipes_sfh(run_name='CNST_SFH_RESOLVED_PBP', bins_to_show=['RESOLVED'], fig=fig, axes=ax, resolved_color='black', time_unit='yr', zorder=10)

In [ ]:
det_galaxy_region = copy.copy(galaxy.det_data["seg"])
            # Value of segmap in center
center = int(galaxy.cutout_size // 2)
possible_vals = np.unique(det_galaxy_region)
if len(possible_vals) == 2:
    det_gal_mask = det_galaxy_region == np.max(possible_vals)
elif len(possible_vals) > 2:
    if int(galaxy.galaxy_id) in possible_vals:
        center_val = int(galaxy.galaxy_id)
        print('Using galaxy ID as center', center_val)
    else:
        center_val = det_galaxy_region[center, center]
mask = det_galaxy_region == center_val
det_gal_mask = np.zeros_like(det_galaxy_region)
det_gal_mask[mask] = True



In [ ]:
galaxy.add_det_galaxy_region(overwrite=True)

In [ ]:
galaxy.pixedfit_processing(gal_region_use='detection', overwrite=True)

In [ ]:
run_dir = '/nvme/scratch/work/tharvey/EXPANSE/pipes'

galaxy.get_total_resolved_property(pxp_name, property='stellar_mass', overwrite=True, run_dir=run_dir, correct_map_band='F444W',
            correct_map=['detection', 'SNR_2_all_wide_nobreak'])

In [ ]:
galaxy.photometry_table['star_stack']['pixedfit'].colnames

In [ ]:
for galaxy in tqdm(galaxies):

    '''
    galaxy.pixedfit_processing(gal_region_use='detection', overwrite=True)
    galaxy.voronoi_binning(SNR_reqs=7, galaxy_region='detection', overwrite=True,
                use_only_widebands=False, plot=False, quiet=True,
                ref_band = 'combined_average')
    galaxy.measure_flux_in_bins(binmap_type='voronoi', overwrite=True)
    '''

    fluxes = []
    #if 'CNST_SFH_RESOLVED_VORONOI' not in galaxy.sed_fitting_table['bagpipes'].keys():
    #    continue
    

    for region in ['pixedfit', 'pixedfit_nomin', 'voronoi']:#, 'pixel_by_pixel']:
        if region not in galaxy.photometry_table['star_stack'].keys():
            continue
       
        mask = galaxy.photometry_table['star_stack'][region]['ID'] == 'TOTAL_BIN'
        flux = galaxy.photometry_table['star_stack'][region][mask]['F444W']
        fluxes.append(flux[0])
        #print(np.sum(galaxy.photometry_table['star_stack'][region]['F444W'][:-4]))

    # Should all be the same within 5%

    if not np.allclose(fluxes, fluxes[0], atol=0.0, rtol=0.05):
        print(galaxy.galaxy_id, fluxes)
    
#[:-4]['F444W'])

In [ ]:
galaxy.measure_flux_in_bins(binmap_type='pixedfit', overwrite=True)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)
galaxy.plot_bagpipes_fit(run_name=pxp_name, bins_to_show=['RESOLVED'],  resolved_color='black', run_dir=run_dir, fig=fig, axes=ax)
galaxy.plot_bagpipes_fit(run_name=resolved_name, bins_to_show=['RESOLVED'],  resolved_color='red', run_dir=run_dir, fig=fig, axes=ax)
ax.set_xlim(0, 5)
ax.set_ylim(32, 23)

In [ ]:
galaxy.plot_bagpipes_results(run_name=pxp_name, binmap_type='pixel_by_pixel', parameters=[
            "stellar_mass",
            "sfr",
            "dust:Av",
            "chisq_phot-",
            "UV_colour",
        ])

In [ ]:
mask = galaxy.gal_region['SNR_2_all_wide_nobreak'].astype(bool)

np.sum(galaxy.psf_matched_data['star_stack']['F444W'][mask])